# Find Diverse set based on Embeddings 

In [87]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
import pandas as pd    
%matplotlib inline

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Creating the input data

In [89]:
df = pd.read_csv(r'/content/drive/MyDrive/SemSeg2/data/leftImg8bit_embed_EXP_1.csv')  # SAME EMBEDDING ALL SAMPLE SLICES IN EXP 1 
df = df.iloc[: , 1:]
df.head(10)

,RawImage,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.014413,0.170611,0.212490,-0.330518,0.321972,0.252612,-0.031215,-0.040536,-0.029646,-0.058093,-0.194263,-0.191752,0.246660,-0.189209,-0.061984,0.065859,0.175657,0.009895,-0.133323,0.151917,0.252837,0.044244,-0.084254,0.233120,-0.243380,0.003560,0.208591,-0.125964,-0.056416,-0.225166,0.195348,-0.229807
1,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.009073,0.166821,0.219175,-0.334357,0.332337,0.239002,-0.049960,-0.039648,-0.036768,-0.047087,-0.183605,-0.207161,0.234042,-0.202094,-0.060192,0.056800,0.166820,0.017448,-0.130760,0.164266,0.253231,0.029426,-0.094874,0.230893,-0.240827,0.015536,0.195171,-0.125420,-0.049672,-0.239202,0.187172,-0.227283
2,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.005397,0.174658,0.211734,-0.337274,0.320230,0.253983,-0.032682,-0.043588,-0.031949,-0.050417,-0.190749,-0.197860,0.242714,-0.197683,-0.055987,0.061593,0.168562,0.007307,-0.133025,0.145428,0.250357,0.039519,-0.079908,0.223275,-0.244257,0.009502,0.209915,-0.126989,-0.056697,-0.230002,0.195572,-0.233986
3,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.011813,0.177237,0.205823,-0.337278,0.309933,0.269490,-0.015331,-0.040652,-0.026104,-0.060639,-0.199665,-0.179700,0.258109,-0.190044,-0.058881,0.063962,0.175229,-0.005613,-0.135240,0.132252,0.251069,0.050836,-0.073321,0.223738,-0.248994,0.003051,0.212864,-0.123788,-0.068463,-0.210509,0.201087,-0.235147
4,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.015960,0.161591,0.226950,-0.324636,0.345087,0.224496,-0.066893,-0.039773,-0.041365,-0.051953,-0.178263,-0.214701,0.217494,-0.193401,-0.067114,0.063726,0.166528,0.037565,-0.122668,0.181547,0.257131,0.029191,-0.110188,0.239281,-0.230118,0.016531,0.196408,-0.125878,-0.035470,-0.254656,0.180481,-0.214002
5,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.031546,0.147326,0.232410,-0.305322,0.360909,0.200242,-0.082137,-0.036525,-0.050980,-0.058737,-0.176700,-0.210832,0.210359,-0.173143,-0.086731,0.070248,0.176635,0.068075,-0.115634,0.208520,0.258795,0.041788,-0.128742,0.258897,-0.212930,0.006421,0.195152,-0.129107,-0.019984,-0.265124,0.167061,-0.198699
6,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.047889,0.102562,0.242324,-0.254106,0.404228,0.095976,-0.156590,-0.028670,-0.088288,-0.051992,-0.131168,-0.237834,0.137600,-0.141677,-0.114568,0.066665,0.159647,0.156999,-0.080054,0.286405,0.252300,0.014146,-0.199756,0.294069,-0.158893,0.024094,0.150052,-0.120783,0.035096,-0.314059,0.114445,-0.142819
7,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.013807,0.168050,0.213373,-0.333556,0.322627,0.250325,-0.032523,-0.041623,-0.031029,-0.053939,-0.193640,-0.190425,0.248907,-0.186865,-0.064821,0.063856,0.176440,0.012644,-0.133033,0.152917,0.253117,0.044500,-0.088003,0.235436,-0.239609,0.003282,0.207272,-0.125584,-0.054192,-0.226912,0.191433,-0.231590
8,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.039760,0.139672,0.232350,-0.299127,0.369368,0.176575,-0.094127,-0.032016,-0.062278,-0.058928,-0.169067,-0.212202,0.202403,-0.161678,-0.095789,0.069565,0.178489,0.087892,-0.110631,0.223793,0.259845,0.037397,-0.147952,0.276452,-0.201996,0.006328,0.185764,-0.129995,-0.008406,-0.273222,0.156542,-0.187751
9,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.006494,0.186859,0.193018,-0.350857,0.286540,0.296520,-0.000178,-0.045021,-0.014221,-0.049913,-0.200337,-0.176884,0.265658,-0.207807,-0.036295,0.054194,0.166830,-0.040391,-0.143799,0.100261,0.243162,0.046101,-0.046868,0.194641,-0.261382,0.010090,0.211047,-0.122023,-0.085899,-0.194758,0.211504,-0.247133


In [90]:
input_features=df.iloc[: , 1:]
input_features.shape

(2975, 32)

# SET SAMPLE SIZE

In [91]:
#Select the total clusters , also the number of samples we want at the end 
S=500    #also change depending on sample slice   !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# CLUSTERING 

In [92]:
#Set Up data sets to be referencing the same random set Cx and Number of randomly chosen clusers S as is applied to random. 
Cx=input_features
#total number clusters is the number of samples we want.
n_clusters=S

#Use best Cluster method that searches sample space , and finds groupings(clusters) that are similar
#The Agglomerative method will stop when the sample size has been reached
#Euclidian distance is used to find most similar groupings and put into the same cluster
#Ward linkage method is used to account of extremities
#Each point in random set is eventually given a cluster , and total number clusters is the number of samples we want.  
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')  
k=cluster.fit_predict(Cx)
Data=pd.DataFrame(Cx)
Cluster=pd.DataFrame(k , columns=['Cluster'])
result = pd.concat([Data, Cluster], axis=1)

#Select Nearest Center

In [93]:
#Many records per cluster , need to find the best distanced points 
#result['Cluster'].value_counts().sort_index(ascending=False).plot(kind="barh" , figsize=(30, 22))

In [94]:
#Find centroid of each cluster 
CenterMatrix=result.groupby("Cluster").agg({'0' : 'mean'  , '1' : 'mean', '2' : 'mean' , '3' : 'mean', '4' : 'mean' , '5' : 'mean', '6' : 'mean' , '7' : 'mean', '8' : 'mean' ,'9' : 'mean', '10' : 'mean' , '11' : 'mean', '12' : 'mean' , '13' : 'mean', '14' : 'mean' , '15' : 'mean', '16' : 'mean' ,'17' : 'mean', '18' : 'mean' , '19' : 'mean', '20' : 'mean' , '21' : 'mean', '22' : 'mean' , '23' : 'mean', '24' : 'mean' ,'25' : 'mean', '26' : 'mean' , '27' : 'mean', '28' : 'mean' , '29' : 'mean', '30' : 'mean' , '31' : 'mean'        })


In [95]:
#Make a center cluster for each of the image rows
H=pd.merge(result, CenterMatrix, how='inner', left_on='Cluster', right_on='Cluster')

In [96]:
#Compute each points distance from its center 


#importing numpy
import numpy as np

#initializing two arrays
array1 = np.array(H.iloc[ : ,33:])
array2 = np.array(result.iloc[0 , :-1 ])

#computing the Euclidan distance 
temp = array1 - array2
distance = np.sqrt(np.sum(np.square(temp)))


print("Euclidean Distance: ", distance)


Euclidean Distance:  33.614210517818215


In [97]:
#initializing two arrays
ls=list()
for i in range(len(H)):
  array1 = np.array(H.iloc[ i:i+1 ,33:]) #center points
  array2 = np.array(result.iloc[i , :-1 ]) #Matrix points
  temp = array1 - array2
  distance = np.sqrt(np.sum(np.square(temp)))
  ls.append([i,distance])

euclid = pd.DataFrame(ls,columns=["rec","distance"])

In [98]:
final = pd.concat([result, euclid], axis=1)


In [99]:
# Select for sample the true point that gives us the minimum distance 
out2=final.loc[final.groupby('Cluster').distance.idxmin()] #true points 

In [100]:
out2.shape

(500, 35)

In [101]:
out2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,Cluster,rec,distance
2367,0.052577,0.010633,0.258109,-0.119533,0.395689,-0.135893,-0.304796,-0.015506,-0.125360,-0.046092,0.026826,-0.271597,-0.078933,-0.059949,-0.123518,0.042775,0.069538,0.290634,0.028108,0.361713,0.177384,-0.065733,-0.271936,0.229658,-0.020208,0.084045,0.031007,-0.051548,0.122310,-0.344594,0.018669,-0.015798,0,2367,0.012702
2354,0.058684,0.004342,0.251160,-0.106911,0.395279,-0.137698,-0.300124,-0.012695,-0.124288,-0.039167,0.022959,-0.261856,-0.075757,-0.054407,-0.129330,0.041719,0.074680,0.293998,0.026734,0.368417,0.177277,-0.053456,-0.275154,0.244703,-0.016742,0.076544,0.032818,-0.057574,0.127580,-0.343565,0.010732,-0.007004,1,2354,0.041701
2800,0.051225,0.025015,0.263587,-0.139535,0.403061,-0.114149,-0.287404,-0.013704,-0.124422,-0.041922,0.005581,-0.274834,-0.051976,-0.078805,-0.118885,0.039513,0.076167,0.277157,0.014622,0.358461,0.188225,-0.063408,-0.270479,0.235160,-0.042820,0.083171,0.043084,-0.064283,0.115214,-0.349787,0.030193,-0.040058,2,2800,0.067309
2280,0.058544,0.014638,0.257285,-0.120462,0.406293,-0.117114,-0.281404,-0.012361,-0.123927,-0.035982,0.002128,-0.265436,-0.046700,-0.073669,-0.129292,0.039825,0.083656,0.283643,0.012002,0.361009,0.193432,-0.049880,-0.273265,0.247838,-0.035393,0.076465,0.043199,-0.065116,0.119679,-0.348624,0.019221,-0.033688,3,2280,0.122874
1913,0.065858,0.019792,0.253460,-0.135744,0.409250,-0.088060,-0.263399,-0.009580,-0.120860,-0.041733,-0.016405,-0.258771,-0.031127,-0.070550,-0.132211,0.043866,0.097089,0.278383,-0.000768,0.363504,0.200540,-0.034912,-0.270621,0.263410,-0.049205,0.067827,0.060752,-0.079451,0.113537,-0.352245,0.027198,-0.033399,4,1913,0.041470
535,0.026022,0.152984,0.232811,-0.312930,0.353860,0.206867,-0.076052,-0.033938,-0.049213,-0.056792,-0.173514,-0.213035,0.211511,-0.181839,-0.079591,0.067071,0.171923,0.058162,-0.116881,0.200090,0.260787,0.034489,-0.120194,0.252477,-0.220248,0.013767,0.195294,-0.126537,-0.025114,-0.260351,0.174952,-0.207779,5,535,0.033651
1494,0.060021,0.076177,0.254563,-0.226441,0.409482,0.041668,-0.186807,-0.020648,-0.094896,-0.043348,-0.101479,-0.247030,0.094284,-0.127869,-0.121245,0.055909,0.148249,0.192590,-0.068625,0.316785,0.245970,0.002250,-0.225637,0.295718,-0.134153,0.038817,0.119726,-0.112935,0.067897,-0.329887,0.095479,-0.111203,6,1494,0.019018
1602,0.049948,0.137783,0.232852,-0.290986,0.372558,0.174758,-0.089999,-0.030554,-0.066003,-0.062635,-0.168151,-0.206500,0.206169,-0.157362,-0.100456,0.070950,0.179714,0.090466,-0.107205,0.227606,0.258711,0.041992,-0.146685,0.280665,-0.202607,0.000754,0.191459,-0.127893,-0.009799,-0.270906,0.154836,-0.188092,7,1602,0.106665
1694,0.036506,0.114932,0.254842,-0.279556,0.398163,0.103890,-0.155206,-0.027710,-0.082303,-0.047092,-0.121867,-0.255769,0.132651,-0.163380,-0.096610,0.054197,0.148054,0.138220,-0.082422,0.270247,0.256218,-0.006194,-0.187771,0.267492,-0.171760,0.048217,0.139673,-0.112957,0.027031,-0.314083,0.135971,-0.164463,8,1694,0.265053
2480,0.068979,0.021203,0.249614,-0.128569,0.411569,-0.079643,-0.265567,-0.014997,-0.116085,-0.043249,-0.025663,-0.253196,-0.027232,-0.067612,-0.139343,0.053375,0.097222,0.279272,-0.002166,0.364615,0.200592,-0.028710,-0.270757,0.270431,-0.049651,0.061049,0.064306,-0.081250,0.107443,-0.351628,0.030200,-0.029271,9,2480,0.022950


In [102]:
#Link it to name of files on index 

selected=pd.merge(df, out2, left_index=True, right_index=True)
selected

,RawImage,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,24_x,25_x,26_x,27_x,28_x,29_x,30_x,31_x,0_y,1_y,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,24_y,25_y,26_y,27_y,28_y,29_y,30_y,31_y,Cluster,rec,distance
0,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.014413,0.170611,0.212490,-0.330518,0.321972,0.252612,-0.031215,-0.040536,-0.029646,-0.058093,-0.194263,-0.191752,0.246660,-0.189209,-0.061984,0.065859,0.175657,0.009895,-0.133323,0.151917,0.252837,0.044244,-0.084254,0.233120,-0.243380,0.003560,0.208591,-0.125964,-0.056416,-0.225166,0.195348,-0.229807,0.014413,0.170611,0.212490,-0.330518,0.321972,0.252612,-0.031215,-0.040536,-0.029646,-0.058093,-0.194263,-0.191752,0.246660,-0.189209,-0.061984,0.065859,0.175657,0.009895,-0.133323,0.151917,0.252837,0.044244,-0.084254,0.233120,-0.243380,0.003560,0.208591,-0.125964,-0.056416,-0.225166,0.195348,-0.229807,31,0,0.009600
7,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.013807,0.168050,0.213373,-0.333556,0.322627,0.250325,-0.032523,-0.041623,-0.031029,-0.053939,-0.193640,-0.190425,0.248907,-0.186865,-0.064821,0.063856,0.176440,0.012644,-0.133033,0.152917,0.253117,0.044500,-0.088003,0.235436,-0.239609,0.003282,0.207272,-0.125584,-0.054192,-0.226912,0.191433,-0.231590,0.013807,0.168050,0.213373,-0.333556,0.322627,0.250325,-0.032523,-0.041623,-0.031029,-0.053939,-0.193640,-0.190425,0.248907,-0.186865,-0.064821,0.063856,0.176440,0.012644,-0.133033,0.152917,0.253117,0.044500,-0.088003,0.235436,-0.239609,0.003282,0.207272,-0.125584,-0.054192,-0.226912,0.191433,-0.231590,339,7,0.012705
14,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.004678,0.173649,0.210461,-0.339641,0.314897,0.264582,-0.027386,-0.040481,-0.026546,-0.052743,-0.193129,-0.190947,0.245044,-0.201494,-0.051104,0.058003,0.168089,-0.003154,-0.137006,0.140452,0.250325,0.039742,-0.075587,0.221164,-0.248367,0.008896,0.208771,-0.124444,-0.062279,-0.222890,0.198275,-0.235403,0.004678,0.173649,0.210461,-0.339641,0.314897,0.264582,-0.027386,-0.040481,-0.026546,-0.052743,-0.193129,-0.190947,0.245044,-0.201494,-0.051104,0.058003,0.168089,-0.003154,-0.137006,0.140452,0.250325,0.039742,-0.075587,0.221164,-0.248367,0.008896,0.208771,-0.124444,-0.062279,-0.222890,0.198275,-0.235403,480,14,0.072427
17,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.013507,0.170153,0.218244,-0.332628,0.333752,0.236685,-0.047898,-0.046039,-0.042114,-0.058319,-0.182955,-0.198294,0.234986,-0.182336,-0.067385,0.066270,0.170709,0.024476,-0.128438,0.163818,0.254426,0.037057,-0.097488,0.237565,-0.232017,0.011082,0.206377,-0.121736,-0.053266,-0.241754,0.186340,-0.230028,0.013507,0.170153,0.218244,-0.332628,0.333752,0.236685,-0.047898,-0.046039,-0.042114,-0.058319,-0.182955,-0.198294,0.234986,-0.182336,-0.067385,0.066270,0.170709,0.024476,-0.128438,0.163818,0.254426,0.037057,-0.097488,0.237565,-0.232017,0.011082,0.206377,-0.121736,-0.053266,-0.241754,0.186340,-0.230028,203,17,0.031895
20,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.015782,0.167978,0.216812,-0.327475,0.331378,0.240392,-0.040909,-0.043036,-0.034706,-0.056046,-0.190425,-0.196236,0.240471,-0.186913,-0.070985,0.064889,0.172937,0.021266,-0.133063,0.166736,0.250523,0.039890,-0.090948,0.240720,-0.234669,0.002363,0.206651,-0.128233,-0.048245,-0.232966,0.188677,-0.228417,0.015782,0.167978,0.216812,-0.327475,0.331378,0.240392,-0.040909,-0.043036,-0.034706,-0.056046,-0.190425,-0.196236,0.240471,-0.186913,-0.070985,0.064889,0.172937,0.021266,-0.133063,0.166736,0.250523,0.039890,-0.090948,0.240720,-0.234669,0.002363,0.206651,-0.128233,-0.048245,-0.232966,0.188677,-0.228417,303,20,0.038946
21,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.011864,0.167900,0.219636,-0.330583,0.335799,0.239643,-0.051447,-0.042061,-0.034829,-0.052253,-0.184283,-0.202093,0.229243,-0.195254,-0.062520,0.063656,0.168347,0.026769,-0.127805,0.166884,0.252251,0.035182,-0.099990,0.234826,-0.2356

#Record Metric

In [103]:
#Total Distance and average distance for this set 

print(out2['distance'].sum())
print(out2['distance'].sum()/S)


75.03768981002007
0.15007537962004014



N100: 5.019604681043471
0.05019604681043471

N200: 


N500:75.03768981002007
0.15007537962004014



N1000 ; 223.02322939360462
0.2230232293936046

N1500 : 417.8197987491535
0.2785465324994357

N2000 : 634.6101022490352
0.3173050511245176

N2500 : 813.6256180923317
0.32545024723693267

# CREATE FINAL DATAFRAME 

In [104]:
#fetch masks file names 
K=pd.DataFrame(selected['RawImage'])
K


,RawImage
0,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
7,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
14,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
17,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
20,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
21,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
27,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
31,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
33,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
45,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...


In [105]:
#Fetch Corrisponding Ground Mask From Meta Data 
import os
os.chdir("/content/drive/MyDrive/SemSeg2/Experiments/CityScapes")

m= pd.read_csv(r'CityScapes_MetaData.csv')
m.head(10)

,Unnamed: 0,ImageName,RawImage,Mask,ColourMask
0,0,aachen_000000_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
1,1,aachen_000001_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
2,2,aachen_000002_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
3,3,aachen_000003_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
4,4,aachen_000004_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
5,5,aachen_000005_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
6,6,aachen_000006_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
7,7,aachen_000007_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
8,8,aachen_000008_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
9,9,aachen_000009_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...


In [106]:
J = pd.merge(K, m, how="inner", on="RawImage")
J[['ImageName' ,	'RawImage','Mask'	, 'ColourMask']]

,ImageName,RawImage,Mask,ColourMask
0,aachen_000000_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
1,aachen_000007_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
2,aachen_000014_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
3,aachen_000017_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
4,aachen_000020_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
5,aachen_000021_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
6,aachen_000027_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
7,aachen_000031_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
8,aachen_000033_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
9,aachen_000045_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...


#CHANGE FILE NAME

In [107]:
#Write to a data frame 

import os
os.chdir("/content/drive/MyDrive/SemSeg2/Experiments/CityScapes/CoreSet_1")


#J.to_csv('CityScapes_Diversity_EXP1_100.csv')
#J.to_csv('CityScapes_Diversity_EXP1_200.csv')
J.to_csv('CityScapes_Diversity_EXP1_500.csv')
#J.to_csv('CityScapes_Diversity_EXP1_1000.csv')
#J.to_csv('CityScapes_Diversity_EXP1_1500.csv')
#J.to_csv('CityScapes_Diversity_EXP1_2000.csv')
#J.to_csv('CityScapes_Diversity_EXP1_2500.csv')

#SHOW VISUALS OF CLUSTERS 

In [76]:
pd.set_option('display.max_rows', None)
k=result[['Cluster']].value_counts()
k

Cluster
141        47
41         39
63         38
50         35
15         34
2          34
28         34
23         34
42         33
59         33
8          33
75         33
44         32
101        32
53         31
16         29
31         29
73         27
21         27
4          27
85         27
128        27
0          26
109        26
7          26
39         25
79         25
54         25
67         25
87         25
9          24
115        23
168        23
110        22
36         21
40         21
35         21
52         21
66         21
5          20
112        20
49         20
64         20
61         20
119        19
20         19
3          18
123        18
120        18
116        18
6          18
58         18
92         18
12         18
32         18
62         18
106        17
94         17
134        17
27         17
84         17
80         17
130        17
99         17
152        17
153        17
169        17
143        17
138        17
24         16
102        1

In [77]:
selectedX=pd.merge(df, result, left_index=True, right_index=True)
selectedX

Output hidden; open in https://colab.research.google.com to view.

In [78]:
view = selectedX[['RawImage','Cluster']]
v=view.loc[view['Cluster']==60]['RawImage'].tolist()
v

['/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000008_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bochum/bochum_000000_024524_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bremen/bremen_000131_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bremen/bremen_000143_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bremen/bremen_000256_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/hamburg/hamburg_000000_047157_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/jena/jena_000003_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/jena/jena_000005_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/jena/jena_000012_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/jena/jena_000082_000019_leftImg8bit.

In [79]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# loads the image in color
img = cv2.imread(v[0])
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [80]:
img = cv2.imread(v[1])
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [81]:
img = cv2.imread(v[2])
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

# POPULATE INTO FOLDERS ---> Clear and add to folder 

MAKE SURE ALLIGNMENT BETWEEN THE IMAGES AND MASKS 

In [108]:
#READ EXPERIMENT FILE 
import pandas as pd
import os
os.chdir("/content/drive/MyDrive/SemSeg2/Experiments/CityScapes/CoreSet_1")

#Same core embedding different clustering seed
#df = pd.read_csv('CityScapes_Diversity_EXP1_100.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_200.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_500.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_1000.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_1500.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_2000.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_2500.csv')  


df=J

r = df['RawImage']
ls_mask = df['Mask']


In [110]:
len(df)

500

In [111]:
#clear the images
mydir = '/content/drive/MyDrive/SemSeg2/data_random/leftImg8bit/train/all'

filelist = [ f for f in os.listdir(mydir) ]
for f in filelist:
  os.remove(os.path.join(mydir, f))

#clear masks 
mydir = '/content/drive/MyDrive/SemSeg2/data_random/gtFine/train/all'

filelist = [ f for f in os.listdir(mydir) ]
for f in filelist:
  os.remove(os.path.join(mydir, f))

In [112]:
#Now copy into a directory into the two respoective folders 
#print(r)

import shutil

#clear all files but not the folder stucture 

for i in range(len(r)):
  shutil.copy(r[i], r'/content/drive/MyDrive/SemSeg2/data_random/leftImg8bit/train/all')
  print(r[i])

/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000007_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000014_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000017_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000020_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000021_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000027_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000031_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000033_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000045_000019_leftImg8bit.png
/content/drive/MyDri

In [113]:
#Now copy into a directory into the two respective folders 
import shutil

#clear all files but not the folder stucture 

for i in range(len(ls_mask)):
  shutil.copy(ls_mask[i], r'/content/drive/MyDrive/SemSeg2/data_random/gtFine/train/all')
  print(ls_mask[i])

/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000000_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000007_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000014_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000017_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000020_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000021_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000027_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000031_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000033_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000045_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2